## Stock Imports

In [173]:
import shutil
import os
import sys
import re

In [174]:
import numpy as np
import scipy
import PIL

In [175]:
from scipy.optimize import minimize_scalar, minimize

In [176]:
import bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Text
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge

print("Bokeh Version:", bokeh.__version__)

output_notebook()
bokeh.io.curdoc().theme = 'dark_minimal'

Bokeh Version: 2.1.1


In [177]:
from bokeh.models import tickers

In [178]:
import bokeh.palettes as palettes

In [179]:
deg = (2*np.pi)/360

## Theoretical Kernel Definitions

In [96]:
K1Target = np.array(
    [[0.400*np.exp(1j*(   0)*deg), 0.311*np.exp(1j*(  40)*deg), 0.222*np.exp(1j*(  80)*deg)],
     [0.400*np.exp(1j*( 120)*deg), 0.311*np.exp(1j*( 160)*deg), 0.222*np.exp(1j*(-160)*deg)],
     [0.400*np.exp(1j*(-120)*deg), 0.311*np.exp(1j*( -80)*deg), 0.222*np.exp(1j*( -40)*deg)]
    ])

In [97]:
K2Target = np.array(
    [[0.467*np.exp(1j*(  26.34)*deg), 0.585*np.exp(1j*( 140.39)*deg), 0.402*np.exp(1j*( -19.92)*deg)],
     [0.614*np.exp(1j*(  34.34)*deg), 0.425*np.exp(1j*( -66.30)*deg), 0.407*np.exp(1j*( -131.93)*deg)],
     [0.358*np.exp(1j*(  14.63)*deg), 0.446*np.exp(1j*(  17.06)*deg), 0.629*np.exp(1j*(  74.63)*deg)]
    ])

In [98]:
K4Target = np.array(
    [[0.606*np.exp(1j*(  44.99)*deg), 0.483*np.exp(1j*( 165.15)*deg)],
     [0.483*np.exp(1j*(  26.71)*deg), 0.606*np.exp(1j*( -33.13)*deg)]
    ])

In [99]:
class TargSParams:

    def __init__(self, kernel, wls):
        self.kernel = kernel
        self.wls = np.array(wls)
        self.rn, self.rc = kernel.shape

    def __getitem__(self, rc):
        if len(rc) == 2:
            r, c = rc
            val = self.kernel[r-1-self.rn, c-1]
            return np.full(len(self.wls), val)
        elif rc == 'wls':
            return self.wls
        else:
            print("I don't know what you want")

In [100]:
K1Targ = TargSParams(K1Target, [1.520, 1.530])

In [101]:
K1Targ[4,1]

array([0.4+0.j, 0.4+0.j])

In [102]:
K1Targ['wls']

array([1.52, 1.53])

## Function Library

### File System

All data is kept in https://github.com/brianedw/SiPhDataStore.  Because data is not persistent in CoLab, it needs to be downloaded in the beginning of the each session.

In [103]:
def reloadSiPhDataStore():
    # Go to the top level
    while(os.getcwd() != '/'):
        os.chdir('..')
    # drop into the content folder.
    os.chdir('content')
    # Delete SiPhDataStore if it exists
    try:
        shutil.rmtree('SiPhDataStore')
    except OSError as e:
        print ("Error: %s - %s." % (e.filename, e.strerror))
    # Download all the files again
    !git clone https://github.com/brianedw/SiPhDataStore.git
    # move to our new data directory
    os.chdir('SiPhDataStore')

### Utility Functions

In [104]:
def dBToLinPower(dbData):
    return 10**(dbData/10)

In [105]:
dBToLinPower(-3)

0.5011872336272722

In [106]:
def find_nearest(array, value):
    array = np.array(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [107]:
find_nearest([0,2,4,6,8], 4.2)

4

In [108]:
def find_nearest_index(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [109]:
find_nearest_index([0,2,4,6,8], 4.2)

2

In [110]:
def matrixDiffMag(m1, m2):
    """
    Computes the magnitude of the difference between two complex matrices
    """
    return (abs(m1 - m2)**2).sum()

In [111]:
def matrixMagDiffMag(m1, m2):
    """
    Computes the magnitude of the difference between two the magnituds of two complex matrices
    """
    return (abs(np.abs(m1) - np.abs(m2))**2).sum()

In [112]:
def matrixDiffVarPh(ph, m1, m2):
    """
    Computes the magnitude of the difference between two complex matrices
    where one one is rotated by ph.
    """
    return (abs(m1 - np.exp(1j*ph)*m2)**2).sum()

In [113]:
def matrixDiffVarPhase(m1, m2):
    """
    Computes the magnitude of the difference between two complex matrices
    where one one is rotated by ph to find the minimul distance.
    """
    soln = minimize_scalar(matrixDiffVarPh, args=(m1, m2), bounds=(-np.pi, np.pi), method='bounded')
    val = soln.fun
    return val

### Fitted Deembedding

In [114]:
def makeCFUniform(theta1, n):
    """
    makes a correction factor of the form
    """
    cf1 = np.exp(1j*(theta1)*deg)
    CF1 = np.full((n,n), cf1)
    CF = CF1 * CF1.T
    return CF

In [115]:
def makeCFSymmetrical(thetaSequence):
    thetaHalfArray = np.array(thetaSequence)
    thetaArray = np.concatenate((thetaHalfArray, thetaHalfArray[-2::-1])).reshape((1,-1))
    CF1 = np.exp(1j*thetaArray*deg)
    CF = CF1 * CF1.T
    return CF

In [116]:
def findSF(pMatA, pMatB):
    def f(sf):
        error = matrixMagDiffMag(pMatA*sf, pMatB)
        return error
    soln = minimize(f,[0])
    arr1D = soln.x
    sf = arr1D[0]
    return sf

In [117]:
def findSingleRotCF(matA, matB):
    def f(arr1D):
        (theta1,) = arr1D
        (n,n) = matA.shape
        CF = makeCFUniform(theta1, n)
        error = matrixDiffMag(matA*CF, matB)
        return error
    soln = minimize(f,[0])
    arr1D = soln.x
    theta1 = arr1D[0]
    print("rotation angle:", theta1)
    (n,n) = matA.shape
    return makeCFUniform(theta1, n)

In [118]:
def findDoubleRotCF(matA, matB):
    # arr1D = [theta1, theta2]
    def f(arr1D):
        (theta1, theta2) = arr1D
        CF = makeCFSymmetrical(arr1D)
        error = matrixDiffMag(matA*CF, matB)
        return error
    soln = minimize(f,[0,0])
    (theta1, theta2) = soln.x
    print("rotation angles:", theta1, theta2)
    return makeCFSymmetrical(soln.x)

In [119]:
a1 = np.array([[1,1,1],[1,1,1],[1,1,1]])
a2 = a1 * makeCFUniform(13., 3)
CF = findSingleRotCF(a1, a2)
matrixDiffMag(a1*CF, a2)

rotation angle: 13.000000126782977


1.7627029018564439e-16

In [120]:
# a1 = np.array([[1,1,1],[1,1,1],[1,1,1]])
# a2 = a1 * makeCF2D(13.,87.)
# CF = findDoubleRotCF(a1, a2)
# matrixDiffMag(a1*CF, a2)

### Importing and Data Containers

In [121]:
def getExpTrace(fName):
    f = open(fName, 'r')
    # Trash the first line
    f.readline()
    # Read the rest
    text = f.read()
    # free the file
    f.close()

    # Data is comma and tab deliminated, remove commas
    text1 = text.replace(',', '')
    # Split on tabs to get a 1D list that is 2*n long
    text2 = text1.split()
    # Convert string data to numbers
    numArrayDB = [float(elem) for elem in text2]
    # Convert to a numpy array and reshape to be n x 2
    npArray = np.array(numArrayDB).reshape((-1,2))
    # Convert to linear scale in power transmission
    dbData = npArray[:, 1]
    linData = dBToLinPower(dbData)
    wls = npArray[:,0]
    return wls, linData

In [122]:
def loadSimResults(fName):
    f = open(fName, 'r')
    # Data consist of three lines
    l1 = f.readline()
    l2 = f.readline()
    l3 = f.readline()
    f.close()

    # First line gives the dimensions of the rest of the data
    nR, nC, nWLs = np.fromstring(l1, dtype=np.uint32, sep='\t')

    # second line gives the wavelengths, which we convert to microns
    wls = (10**6)*np.fromstring(l2, dtype=np.float, sep='\t')
    assert len(wls) == nWLs, 'wavelength data has unexpected length'

    # third line gives the rest of the data.
    textLine = l3
    # Python iterprets 'j' as the imaginary unit
    textLine = textLine.replace('i', 'j')
    # Values are tab separated.  Convert to list of strings.
    textList = textLine.split(sep='\t')
    # Convert to complex values.
    numberList = [complex(w) for w in textList]
    # Convert to numpy array
    npArray = np.array(numberList)
    # Check length makes sense.
    assert len(npArray) == nC*nR*nWLs, 'data not read to expected length'
    # Reshape array
    npArray = npArray.reshape(nWLs, nR, nC)
    # and return.
    print(fName, ':', npArray.shape)
    return wls, npArray

In [123]:
class SimSParams:
    def __init__(self, fName):
        self.wls, self.SDataRaw = loadSimResults(fName)
        self.SData = self.SDataRaw.copy()
        self.nWLs, self.nR, self.nC = self.SData.shape

    def getSTrace(self, r, c):
        return self.SData[:, r-1, c-1]
   
    def getPTrace(self, r, c):
        return np.abs(self.SData[:, r-1, c-1])**2

    def getSVal(self, r, c, wl):
        iWL = find_nearest_index(self.wls, wl)
        return self.SData[iWL, r-1, c-1]

    def getPVal(self, r, c, wl):
        iWL = find_nearest_index(self.wls, wl)
        return np.abs(self.SData[iWL, r-1, c-1])**2

    def getSTransPart(self):
        return self.SData[:, (self.nR//2):, :(self.nC//2) ]

    def getSTransPartAtWL(self, wl):
        iWL = find_nearest_index(self.wls, wl)
        return self.SData[iWL, (self.nR//2):, :(self.nC//2) ]

    def getPTransPart(self):
        return self.SData[:, (self.nR//2):, :(self.nC//2) ]

    def getPTransPartAtWL(self, wl):
        iWL = find_nearest_index(self.wls, wl)
        return np.abs(self.SData[iWL, (self.nR//2):, :(self.nC//2) ])**2

    def applyCorrectionFactor(self, CF):
        zArray = np.zeros_like(CF)
        CFBig = np.block([[[zArray, CF.T],
                           [CF, zArray]]])
        self.SData =  CFBig * self.SDataRaw

    def resetCorrectionFactor(self):
        self.SData = self.SDataRaw


In [124]:
class TargSParams:
    def __init__(self, transArray):
        zArray = np.zeros_like(transArray)
        self.SDataRaw = np.block([[zArray, transArray.T],
                          [transArray, zArray]])
        self.SData = self.SDataRaw.copy()
        self.nR, self.nC = self.SData.shape

    def getSVal(self, r, c):
        return self.SData[r-1, c-1]

    def getPVal(self, r, c):
        return np.abs(self.SData[r-1, c-1])**2

    def getSTransPart(self):
        return self.SData[(self.nR//2):, :(self.nC//2) ]

    def getPTransPart(self):
        return np.abs(self.SData[(self.nR//2):, :(self.nC//2) ])**2
    
    def applyCorrectionFactor(self, CF):
        self.SData = CF * self.SDataRaw

    def resetCorrectionFactor(self):
        self.SData = self.SDataRaw

In [125]:
class ExpSParams:
    def __init__(self, transDict, scaleFactor):
        self.sf = scaleFactor
        n = int(np.sqrt(len(transDict)))
        PKeyArray = [['T'+str(i+1+n)+str(j+1) for j in range(n)] for i in range(n)]
        transArray = np.array([[transDict[key] for key in row] for row in PKeyArray])
        zArray = np.zeros_like(transArray)
        self.PData = np.block([[zArray, transArray.T],
                               [transArray, zArray]])
        self.nR, self.nC = self.PData.shape

    def getPVal(self, r, c):
        return (self.sf)*self.PData[r-1, c-1]

    def getPTransPart(self):
        return (self.sf)*(self.PData[(self.nR//2):, :(self.nC//2) ])

    def applyCorrectionFactor(self, sf):
        self.sf = sf

    def resetCorrectionFactor(self):
        self.sf = 1

In [126]:
class ExpInterParams:
    def __init__(self, transDict, scaleFactor):
        self.sf = scaleFactor
        self.transDict = transDict

    def getPVal(self, rPair, t):
        r1, r2 = rPair
        return (self.sf)*self.transDict['T'+str(r1)+str(r2)+','+str(t)]

    def applyCorrectionFactor(self, sf):
        self.sf = sf

    def resetCorrectionFactor(self):
        self.sf = 1

### Plotting

#### Polar Error Plot

In [127]:
def makePolarPlot(title):
    '''
    This will create a Bokeh plot that depicts the unit circle.

    Requires import bokeh
    '''
    p = bokeh.plotting.figure(plot_width=400, plot_height=400, title=title, 
                              x_range=[-1.1, 1.1], y_range=[-1.1, 1.1])
    p.xaxis[0].ticker=bokeh.models.tickers.FixedTicker(ticks=np.arange(-1, 2, 0.25))
    p.yaxis[0].ticker=bokeh.models.tickers.FixedTicker(ticks=np.arange(-1, 2, 0.25)) 
    p.circle(x = [0,0,0,0], y = [0,0,0,0], radius = [0.25, 0.50, 0.75, 1.0], 
             fill_color = None, line_color='gray')
    p.line(x=[0,0], y=[-1,1], line_color='gray')
    p.line(x=[-1,1], y=[0,0], line_color='gray')
    xs = [0.25, 0.50, 0.75, 1.00]
    ys = [0, 0, 0, 0]
    texts = ['0.25', '0.50', '0.75', '1.00']
    source = bokeh.models.ColumnDataSource(dict(x=xs, y=ys, text=texts))
    textGlyph = bokeh.models.Text(x="x", y="y", text="text", angle=0.3, 
                                  text_color="gray", text_font_size='10px')
    p.add_glyph(source, textGlyph)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p

In [128]:
def addMatrixDiff(bokehPlot, m1, m2):
    """
    This will draw lines showing the difference between two 2D matrices.
    """
    p = bokehPlot
    begX = (np.real(m1)).flatten()
    begY = (np.imag(m1)).flatten()
    endX = (np.real(m2)).flatten()
    endY = (np.imag(m2)).flatten()

    xs = np.array([begX, endX]).T.tolist()
    ys = np.array([begY, endY]).T.tolist()
    p.multi_line()
    p.multi_line(xs=xs, ys=ys)

    sourceTarg = bokeh.models.ColumnDataSource(dict(x=begX.tolist(), y=begY.tolist()))
    glyphTarg = bokeh.models.Circle(x="x", y="y", size=10, line_color="green", 
                                    fill_color=None, line_width=3)
    p.add_glyph(sourceTarg, glyphTarg)

    sourceSim = bokeh.models.ColumnDataSource(dict(x=endX.tolist(), y=endY.tolist()))
    glyphSim = bokeh.models.Circle(x="x", y="y", size=5, line_color=None, 
                                   fill_color='red', line_width=3)
    p.add_glyph(sourceSim, glyphSim)
    return p

#### Phase Error Simulation Plot

In [129]:
def makePhErrorSimPlot(KName, KTarg, KSim):
    KTarget = KTarg.getSTransPart()
    p = figure(plot_width=800, plot_height=400, title=KName + " Sim Error vs Wavelength", x_range=[1.4, 1.6], y_range=[-0.1, 6.1])

    errorsComp = [matrixDiffMag(k, KTarget) for k in KSim.getSTransPart()]
    errorsMag = [matrixMagDiffMag(k, KTarget) for k in KSim.getSTransPart()]
    errorsMagVarPh = [matrixDiffVarPhase(k, KTarget) for k in KSim.getSTransPart()]

    p.line(x=KSim.wls, y=errorsComp, line_color=palettes.Category10[9][0], legend_label="Complex Error")
    p.line(x=KSim.wls, y=errorsMag, line_color=palettes.Category10[9][1], legend_label="Mag Error")
    p.line(x=KSim.wls, y=errorsMagVarPh, line_color=palettes.Category10[9][2], legend_label="Complex Error var Phase")
    return p

#### Dispersion Plot

In [130]:
def makeDispersionPlot(KName, yMax):
    p = figure(plot_width=850, plot_height=400, title=KName+' Single Power Transmissions', x_range=[1.400,1.600], y_range=[0, yMax])
    # p.update_layout(shapes=[dict(type= 'line', yref= 'paper', y0= 0, y1= 1, xref= 'x', x0= 1.525, x1= 1.525)])
    p.xaxis.axis_label = 'wavelength (um)'
    p.yaxis.axis_label = 'T'
    return p

In [131]:
def addSimTraces(p, KSim, rList, tList):
    colorIndex = 0
    for t in tList:
        for r in rList:
            color = palettes.Category10[9][colorIndex]
            colorIndex += 1
            trace = KSim.getPTrace(r,t)
            p.line(KSim.wls, trace, line_color=color, line_width=2, legend_label="abs(S"+str(r)+str(t)+")^2")

In [132]:
def addTargDots(p, KTarg, rList, tList):
    colorIndex = 0
    for t in tList:
        for r in rList:
            color = palettes.Category10[9][colorIndex]
            colorIndex += 1
            val = KTarg.getPVal(r,t)
            p.circle([1.525], [val], size=10, color=color, fill_alpha=0)

In [133]:
def addExpDots(p, KExp, rList, tList):
    colorIndex = 0
    for t in tList:
        for r in rList:
            color = palettes.Category10[9][colorIndex]
            colorIndex += 1
            val = KExp.getPVal(r,t)
            p.cross([1.525], [val], size=10, color=color)

#### Interference Dispersion Plot

In [134]:
def makeInterDispersionPlot(KName, yMax):
    p = figure(plot_width=850, plot_height=400, title=KName+' Interferred Power Transmissions', x_range=[1.400,1.600], y_range=[0, yMax])
    p.xaxis.axis_label = 'wavelength (um)'
    p.yaxis.axis_label = 'T'
    return p

In [135]:
def addSimTraces1(p, KSim, rPairs, tList):
    cIndex = 0
    for t in tVals:
        for rPair in rPairs:
            color = palettes.Category10[9][cIndex]
            cIndex += 1
            r1, r2 = rPair
            trace = np.abs(KSim.getSTrace(r1,t) + KSim.getSTrace(r2,t))**2
            p.line( KSim.wls, trace, line_color=color, line_width=2, legend_label="P"+str(r1)+str(r2)+','+str(t))

In [136]:
def addTargDots1(p, KTarg, rPairs, tList):
    cIndex = 0
    for t in tVals:
        for rPair in rPairs:
            (r1, r2) = rPair
            color = palettes.Category10[9][cIndex]
            cIndex += 1
            val = np.abs(KTarg.getSVal(r1, t) + KTarg.getSVal(r2, t))**2
            p.circle([1.525], [val], size=10, color=color, fill_alpha=0)

In [137]:
def addExpDots1(p, KExpInt, rPairs, tList):
    cIndex = 0
    for t in tVals:
        for rPair in rPairs:
            color = palettes.Category10[9][cIndex]
            cIndex += 1
            val = KExpInt.getPVal(rPair, t)
            p.cross([1.525], [val], size=10, color=color)

#### Power Bar Plot

In [138]:
def MakePowerBarPlot(KTarg, KSim, KExp, rVals, tVals):
    cats = ['T'+str(r)+str(t) for r in rVals for t in tVals]
    subCats = ['targ', 'sim', 'exp']
    dodges = [-0.25, 0.0, 0.25]
    colors = ["#c9d9d3", "#718dbf", "#e84d60"]

    targData = KTarg.getPTransPart().flatten().tolist()
    simData = KSim.getPTransPartAtWL(1.525).flatten().tolist()
    expData = KExp.getPTransPart().flatten().tolist()

    data = {'cats' : cats,
            'targ' : targData,
            'sim' : simData,
            'exp' : expData}
    source = ColumnDataSource(data=data)

    max = np.max((targData,simData,expData))
    p = figure(x_range=cats, y_range=(0, 1.2*max), plot_width=850, plot_height = 300, 
               title="Power Comparisons", toolbar_location=None, tools="")

    for i in range(len(subCats)):
        p.vbar(x=dodge('cats', dodges[i], range=p.x_range), top=subCats[i], width=0.2, source=source,
        color=colors[i], legend_label=subCats[i])

    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    return p

In [139]:
def MakeInterPowerBarPlot(KTarg, KSim, KExpInt, rPairs, tVals):
    cats = ['T'+str(r[0])+str(r[1])+','+str(t) for r in rPairs for t in tVals]
    subCats = ['targ', 'sim', 'exp']
    dodges =  [ -0.25,   0.0,  0.25]
    colors = ["#c9d9d3", "#718dbf", "#e84d60"]

    targData = [np.abs(KTarg.getSVal(r[0],t) + KTarg.getSVal(r[1],t) )**2 for r in rPairs for t in tVals]
    simData = [np.abs(KSim.getSVal(r[0],t, 1.525) + KSim.getSVal(r[1],t, 1.525) )**2 for r in rPairs for t in tVals]
    expData = [KExpInt.getPVal(r, t) for r in rPairs for t in tVals]
    
    data = {'cats' : cats,
            'targ' : targData,
            'sim' : simData,
            'exp' : expData}
    source = ColumnDataSource(data=data)

    max = np.max((targData,simData,expData))
    p = figure(x_range=cats, y_range=(0, 1.2*max), plot_width=850, plot_height = 300, 
                title="Power Comparisons", toolbar_location=None, tools="")

    for i in range(len(subCats)):
        p.vbar(x=dodge('cats', dodges[i], range=p.x_range), top=subCats[i], width=0.2, source=source,
        color=colors[i], legend_label=subCats[i])

    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    return p

# Application

In [140]:
reloadSiPhDataStore()

Cloning into 'SiPhDataStore'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 88 (delta 0), reused 0 (delta 0), pack-reused 77
Unpacking objects: 100% (88/88), done.


## K1 Analysis

In [141]:
KSim = K1Sim
KTarget = K1Target.conj()*np.exp(1j*(20)*deg)
KName = 'K1'

NameError: ignored

In [ ]:
ind = find_nearest_index(KSim['wls'], 1.525)
STransAtWL = KSim.getTransPart()[ind]
wl = round(KSim['wls'][ind],4)

In [ ]:
KTarget

In [ ]:
p = makePolarPlot(KName+' '+str(wl)+'(um)')
addMatrixDiff(p, KTarget, STransAtWL)
show(p)

In [ ]:
errorsComp = [matrixDiffMag(k, KTarget) for k in KSim.getTransPart()]
errorsMag = [matrixMagDiffMag(k, KTarget) for k in KSim.getTransPart()]
errorsMagVarPh = [matrixDiffVarPhase(k, KTarget) for k in KSim.getTransPart()]

In [ ]:
p = figure(plot_width=800, plot_height=400, title="Sim Error vs Wavelength", x_range=[1.4, 1.6], y_range=[-0.1, 3.1])
p.line(x=KSim.wls, y=errorsComp, line_color=palettes.Category10[9][0], legend_label="Complex Error")
p.line(x=KSim.wls, y=errorsMag, line_color=palettes.Category10[9][1], legend_label="Mag Error")
p.line(x=KSim.wls, y=errorsMagVarPh, line_color=palettes.Category10[9][2], legend_label="Complex Error var Phase")
show(p)

In [ ]:
ff = 1.0
p = figure(plot_width=850, plot_height=400, title=KName+' Single Power Transmissions', x_range=[1.400,1.600], y_range=[0, 0.2])
# p.update_layout(shapes=[dict(type= 'line', yref= 'paper', y0= 0, y1= 1, xref= 'x', x0= 1.525, x1= 1.525)])
p.xaxis.axis_label = 'wavelength (um)'
p.yaxis.axis_label = 'T'
for t in [1,2,3]:
    color = palettes.Category10[9][t]
    for r in [4,5,6]:
        p.line( KSim['wls'], np.abs(ff*KSim[r,t])**2, line_color=color, line_width=2, legend_label="abs(S"+str(r)+str(t)+")^2")
        val = np.abs(KTarget[r-3-1,t-1])**2
        p.circle([1.525], [val], size=10, color=color)
for t in [1,2,3]:
    color = palettes.Category10[9][t]
    for r in [4,5,6]:
        dataFName = KName+'_'+'T'+str(r)+str(t)
        try:
            wlsExp, valsExp = getExpTrace(dataFName)
            p.line( wlsExp, valsExp, line_color=color, line_width=2, line_dash='dashed')
        except:
            pass
show(p)

In [ ]:
rPairs = [(4,5), (5,6), (4,6)]
p = figure(plot_width=850, plot_height=400, title=KName + ' Interferred Transmission Powers', x_range=[1.400,1.600], y_range=[0, 0.15])
p.xaxis.axis_label = 'wavelength (um)'
p.yaxis.axis_label = 'T'
cIndex = 0
for t in [1,2,3]:
    for rPair in rPairs:
        color = palettes.Category10[9][cIndex]
        cIndex += 1
        r1, r2 = rPair
        p.line( KSim['wls'], np.abs(KSim[r1,t] + KSim[r2,t])**2, line_color=color, line_width=2, legend_label="abs(S"+str(r1)+str(t)+" + S"+str(r2)+str(t)+")^2")
for t in [1,2,3]:
    color = palettes.Category10[9][t]
    for rPair in rPairs:
        r1, r2 = rPair
        dataFName = KName+'_'+'T'+str(r1)+str(t)+'_T'+str(r1)+str(t)
        try:
            wlsExp, valsExp = getExpTrace(dataFName)
            p.line( wlsExp, valsExp, line_color=color, line_width=2, line_dash='dashed')
        except:
            pass
show(p)

## K2 Analysis

In [ ]:
KSim = K2Sim
KTarget = K2Target.conj() * np.exp(1j*0*deg)
KName = 'K2'

In [ ]:
ind = find_nearest_index(KSim['wls'], 1.525)
STransAtWL = KSim.getTransPart()[ind]
wl = round(KSim['wls'][ind],4)

In [ ]:
p = makePolarPlot(KName+' '+str(wl)+'(um)')
addMatrixDiff(p, KTarget, STransAtWL)
show(p)

In [ ]:
errorsComp = [matrixDiffMag(k, KTarget) for k in KSim.getTransPart()]
errorsMag = [matrixMagDiffMag(k, KTarget) for k in KSim.getTransPart()]
errorsMagVarPh = [matrixDiffVarPhase(k, KTarget) for k in KSim.getTransPart()]

In [ ]:
p = figure(plot_width=800, plot_height=400, title="Sim Error vs Wavelength", x_range=[1.4, 1.6], y_range=[-0.1, 6.1])
p.line(x=KSim.wls, y=errorsComp, line_color=palettes.Category10[9][0], legend_label="Complex Error")
p.line(x=KSim.wls, y=errorsMag, line_color=palettes.Category10[9][1], legend_label="Mag Error")
p.line(x=KSim.wls, y=errorsMagVarPh, line_color=palettes.Category10[9][2], legend_label="Complex Error var Phase")
show(p)

In [ ]:
ff = 1.0
p = figure(plot_width=850, plot_height=400, title=KName+' Single Power Transmissions', x_range=[1.400,1.600], y_range=[0, 0.45])
# p.update_layout(shapes=[dict(type= 'line', yref= 'paper', y0= 0, y1= 1, xref= 'x', x0= 1.525, x1= 1.525)])
cIndex = 0
for t in [1,2,3]:
    for r in [4,5,6]:
        color = palettes.Category10[9][cIndex]
        cIndex +=1
        p.line( KSim['wls'], np.abs(ff*KSim[r,t])**2, line_color=color, line_width=2, legend_label="abs(S"+str(r)+str(t)+")^2")
        val = np.abs(KTarget[r-3-1,t-1])**2
        p.circle([1.525], [val], size=10, color=color)
p.xaxis.axis_label = 'wavelength (um)'
p.yaxis.axis_label = 'T'
show(p)

In [ ]:
rPairs = [(4,5), (5,6), (4,6)]
p = figure(plot_width=850, plot_height=400, title=KName + ' Interferred Transmission Powers', x_range=[1.400,1.600], y_range=[0, 0.60])
# p.update_layout(shapes=[dict(type= 'line', yref= 'paper', y0= 0, y1= 1, xref= 'x', x0= 1.525, x1= 1.525)])
cIndex = 0
for t in [1,2,3]:
    for rPair in rPairs:
        color = palettes.Category10[9][cIndex]
        cIndex += 1
        r1, r2 = rPair
        p.line( KSim['wls'], np.abs(KSim[r1,t] + KSim[r2,t])**2, line_color=color, line_width=2, legend_label="abs(S"+str(r1)+str(t)+" + S"+str(r2)+str(t)+")^2")
p.xaxis.axis_label = 'wavelength (um)'
p.yaxis.axis_label = 'T'
show(p)

## K3 Analysis

In [142]:
K3Exp1525PD = {'T41':  55, 'T42': 107, 'T43':  63,
                'T51': 126, 'T52':  39, 'T53':  59,
                'T61':  46, 'T62':  49, 'T63': 105}

In [143]:
K3Exp1525PDInt = {'T45,1':110, 'T45,2':125, 'T45,3': 70,
                  'T56,1':120, 'T56,2': 36, 'T56,3':135,
                  'T46,1': 99, 'T46,2': 75, 'T46,3':121}

In [144]:
K3Target = np.array(
    [[0.5494*np.exp(1j*(  26.34)*deg), 0.6882*np.exp(1j*( 140.39)*deg), 0.4729*np.exp(1j*( -19.92)*deg)],
     [0.7224*np.exp(1j*(  34.34)*deg), 0.5000*np.exp(1j*( -66.30)*deg), 0.4788*np.exp(1j*( -131.93)*deg)],
     [0.4212*np.exp(1j*(  14.63)*deg), 0.5247*np.exp(1j*(  17.06)*deg), 0.7400*np.exp(1j*(  74.63)*deg)]
    ])

In [145]:
KSim2D = SimSParams('K3_2DEIA_SIM3.txt')
KSim = SimSParams('K3_SIM3.txt')
KTarg = TargSParams(K3Target.conj())
KExp = ExpSParams(K3Exp1525PD, 1.0)
KExpInt = ExpInterParams(K3Exp1525PDInt, 1.0)
KName = 'K3'

K3_2DEIA_SIM3.txt : (201, 6, 6)
K3_SIM3.txt : (201, 6, 6)


In [146]:
KSim2D.resetCorrectionFactor()
CF = findSingleRotCF(KSim2D.getSTransPartAtWL(1.525), KTarg.getSTransPart())

rotation angle: -56.10246521100576


In [147]:
KSim2D.applyCorrectionFactor(CF)

In [148]:
p = makePolarPlot(KName)
addMatrixDiff(p, KTarg.getSTransPart(), KSim2D.getSTransPartAtWL(1.525))
show(p)

In [149]:
p = makePhErrorSimPlot(KName, KTarg, KSim2D)
show(p)

In [150]:
KSim.resetCorrectionFactor()
CF = findDoubleRotCF(KSim.getSTransPartAtWL(1.525), KTarg.getSTransPart())

rotation angles: 16.245013288781557 -99.16776611904042


In [151]:
KSim.applyCorrectionFactor(CF)

In [152]:
p = makePolarPlot(KName)
addMatrixDiff(p, KTarg.getSTransPart(), KSim.getSTransPartAtWL(1.525))
show(p)

In [153]:
p = makePhErrorSimPlot(KName, KTarg, KSim)
show(p)

In [154]:
KSim.resetCorrectionFactor()

In [155]:
KExp.resetCorrectionFactor()
sf = findSF(KExp.getPTransPart(), KSim.getPTransPartAtWL(1.525))
KExp.applyCorrectionFactor(sf)

In [156]:
p = makeDispersionPlot(KName, 0.6)
addSimTraces(p, KSim, (4,5,6), (1,2,3))
addTargDots(p, KTarg, (4,5,6), (1,2,3))
addExpDots(p, KExp, (4,5,6), (1,2,3))
show(p)

In [157]:
p = MakePowerBarPlot(KTarg, KSim, KExp, (4, 5, 6), (1, 2, 3))
show(p)

In [158]:
sf = findSF(KExp.getPTransPart(), KSim.getPTransPartAtWL(1.525))

In [159]:
KExpInt.applyCorrectionFactor(0.0045)

In [160]:
rPairs = [(4,5), (5,6), (4,6)]
tVals = [1,2,3]
p = makeInterDispersionPlot(KName, 1.5)
addSimTraces1(p, KSim, rPairs, tVals)
addTargDots1(p, KTarg, rPairs, tVals)
addExpDots1(p, KExpInt, rPairs, tVals)
show(p)

In [161]:
p = MakeInterPowerBarPlot(KTarg, KSim, KExpInt, ((4,5), (5,6), (4,6)), (1,2,3))
show(p)

## K4

In [206]:
K4Exp1525PD = {'T31':  55, 'T32': 107, 
                'T41': 126, 'T42':  39}

In [207]:
K4Exp1525PDInt = {'T34,1':110, 'T34,2':125}

In [208]:
K4Target = np.array(
    [[0.606*np.exp(1j*(  44.99)*deg), 0.483*np.exp(1j*( 165.15)*deg)],
     [0.483*np.exp(1j*(  26.71)*deg), 0.606*np.exp(1j*( -33.13)*deg)]
    ])

In [209]:
KSim2D = SimSParams('K4_2DEIA_SIM3.txt')
KSim = SimSParams('K4_SIM3.txt')
KTarg = TargSParams(K4Target.conj())
KExp = ExpSParams(K4Exp1525PD, 1.0)
KExpInt = ExpInterParams(K4Exp1525PDInt, 1.0)
KName = 'K4'

K4_2DEIA_SIM3.txt : (201, 4, 4)
K4_SIM3.txt : (201, 4, 4)


In [210]:
KSim2D.SData.shape

(201, 4, 4)

In [211]:
KSim2D.getSTransPartAtWL(1.525)

array([[ 0.545248 +0.116223j, -0.146521 -0.426725j],
       [ 0.372025 +0.215859j, -0.0151761+0.584842j]])

In [212]:
KTarg.getSTransPart().shape

(2, 2)

In [213]:
KSim2D.resetCorrectionFactor()
CF = findSingleRotCF(KSim2D.getSTransPartAtWL(1.525), KTarg.getSTransPart())

rotation angle: -28.625060798524917


In [214]:
KSim2D.applyCorrectionFactor(CF)

In [215]:
p = makePolarPlot(KName)
addMatrixDiff(p, KTarg.getSTransPart(), KSim2D.getSTransPartAtWL(1.525))
show(p)

In [190]:
p = makePhErrorSimPlot(KName, KTarg, KSim2D)
show(p)

In [191]:
KSim.resetCorrectionFactor()
CF = findSingleRotCF(KSim.getSTransPartAtWL(1.525), KTarg.getSTransPart())

rotation angle: -15.534260264750488


In [192]:
KSim.applyCorrectionFactor(CF)

In [193]:
p = makePolarPlot(KName)
addMatrixDiff(p, KTarg.getSTransPart(), KSim.getSTransPartAtWL(1.525))
show(p)

In [194]:
p = makePhErrorSimPlot(KName, KTarg, KSim)
show(p)

In [195]:
KSim.resetCorrectionFactor()

In [196]:
KExp.resetCorrectionFactor()
sf = findSF(KExp.getPTransPart(), KSim.getPTransPartAtWL(1.525))
KExp.applyCorrectionFactor(sf)

In [197]:
p = makeDispersionPlot(KName, 0.6)
addSimTraces(p, KSim, (3,4), (1,2))
addTargDots(p, KTarg, (3,4), (1,2))
addExpDots(p, KExp, (3,4), (1,2))
show(p)

In [198]:
p = MakePowerBarPlot(KTarg, KSim, KExp, (3,4), (1, 2))
show(p)

In [199]:
sf = findSF(KExp.getPTransPart(), KSim.getPTransPartAtWL(1.525))

In [200]:
KExpInt.applyCorrectionFactor(0.0045)

In [201]:
rPairs = [(3,4)]
tVals = [1,2]
p = makeInterDispersionPlot(KName, 1.5)
addSimTraces1(p, KSim, rPairs, tVals)
addTargDots1(p, KTarg, rPairs, tVals)
addExpDots1(p, KExpInt, rPairs, tVals)
show(p)

In [202]:
p = MakeInterPowerBarPlot(KTarg, KSim, KExpInt, ((3,4),), (1,2))
show(p)

# Scrap

In [216]:
getExpTrace("/content/SiPhDataStore/K4_V2/K4T31_n15dBm_44d5K.csv")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in power
  


(array([14710.025288, 14730.025042, 14750.025342, 14770.027077,
        14790.03041 , 14810.03464 , 14830.03868 , 14850.03959 ,
        14870.0391  , 14890.04122 , 14910.04433 , 14930.04743 ,
        14950.05115 , 14970.05275 , 14990.05285 , 15010.05494 ,
        15030.05823 , 15050.06027 , 15070.0613  , 15090.06417 ,
        15110.06657 , 15130.06817 , 15150.0705  , 15170.07148 ,
        15190.06799 , 15210.06677 , 15230.07045 , 15250.07351 ,
        15270.07499 , 15290.07673 , 15310.07591 , 15330.07393 ,
        15350.07231 , 15370.07158 , 15390.06856 , 15410.06566 ,
        15430.06505 , 15450.06291 , 15470.05944 , 15490.05539 ,
        15510.05268 , 15530.04761 , 15550.04411 , 15570.04212 ,
        15590.03978 , 15610.03441 , 15630.03036 , 15650.026261,
        15670.022198, 15690.018628, 15710.016643, 15730.015167,
        15750.013651, 15770.012324, 15790.011062]),
 array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, 

In [217]:
import pandas as pd
df=pd.read_csv("/content/SiPhDataStore/K4_V2/K4T31_n15dBm_44d5K.csv", sep=',',header=None)

In [220]:
K4T31 = np.array(df)

## Bar Charts

In [ ]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure

In [ ]:
cats = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
subCats = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (cat, subCat) for cat in cats for subCat in subCats ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar(x='x', top='counts', width=0.9, source=source)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

In [ ]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge

In [ ]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries'] # cats
years = ['2015', '2016', '2017'] # subCats

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

In [ ]:
keys = list(K3Exp1525PD.keys())

In [ ]:
keys

In [ ]:
rVals = (4, 5, 6)
tVals = (1, 2, 3)

In [ ]:
cats = ['T'+str(r)+str(t) for r in rVals for t in tVals]

In [ ]:
test = KTarg.getPTransPart().flatten().tolist()

In [ ]:
test.tolist()